In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Tabela usadas para análise do pré processamento

- Para fazer análises de como pré processar os dados, como por exemplo, decidir as melhores faixas de playerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayerplayer'playerAge, podemos usar uma tabela com todas as transferências na europa a partir de 2017, que é a tabela 'detailed_transfers' abaixo

In [3]:
transfers = pd.read_csv('data/transfers/all_transfers.csv')
all_players = pd.read_csv('data/players/all_players.csv')
recent_transfers = transfers[transfers['season']>=2017]
detailed_transfers = pd.merge(recent_transfers, all_players[['playerLink', 'currAge', 'height', 'foot']], on="playerLink", how="left")

detailed_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,clubJoinedUrl,teamTransferType,season,valuation,currAge,height,foot
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,/fc-chelsea/startseite/verein/631,in,2017,6200000,22.0,1.82,right
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,in,2017,-27000000,27.0,1.87,right
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,in,2017,14500000,25.0,1.87,right
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,/manchester-city/startseite/verein/281,in,2017,-39700000,33.0,1.83,right
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,/manchester-city/startseite/verein/281,in,2017,0,29.0,1.88,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,Ibrahim Cissoko,/ibrahim-cissoko/profil/spieler/575693,20,LW,Netherlands,2500000,3000000,NEC Nijmegen,/nec-nijmegen/startseite/verein/467,FC Toulouse,/fc-toulouse/startseite/verein/415,in,2023,-500000,20.0,NaN,right
3987,Mahdi Camara,/mahdi-camara/profil/spieler/324795,25,CM,France,3000000,3000000,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,Stade Brestois 29,/stade-brest-29/startseite/verein/3911,in,2023,0,25.0,1.78,right
3988,Mostafa Mohamed,/mostafa-mohamed/profil/spieler/462348,25,CF,Egypt,6000000,5750000,Galatasaray,/galatasaray-istanbul/startseite/verein/141,FC Nantes,/fc-nantes/startseite/verein/995,in,2023,250000,25.0,1.85,right
3989,Andy Delort,/andy-delort/profil/spieler/122797,31,CF,Algeria,7000000,5000000,OGC Nice,/ogc-nizza/startseite/verein/417,FC Nantes,/fc-nantes/startseite/verein/995,in,2023,2000000,31.0,1.81,right


In [56]:
# Passo 1: Carregar os dados em um DataFrame
df = detailed_transfers

# Divida o DataFrame em variáveis independentes (X) e variável dependente (y)
X = df[['playerAge']]
y = df['transferFee']

# Crie um modelo de regressão linear com a idade não processada
model_original = LinearRegression()
model_original.fit(X, y)

# Avalie o desempenho do modelo original
score_original = model_original.score(X, y)
print(f"Score do modelo original: {score_original}")

# Crie dummies para faixas de idade
age_bins = [0, 16, 18, 22, 26, 28, np.inf]
age_labels = ['youngest', 'younger', 'young', 'medium', 'old', 'older']
df['ageCategory'] = pd.cut(df['playerAge'], bins=age_bins, labels=age_labels)

# Crie dummies a partir da coluna 'ageCategory'
transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [0])], remainder='passthrough')
X_encoded = transformer.fit_transform(df[['ageCategory']])

# Crie um modelo de regressão linear com dummies de faixas de idade
model_encoded = LinearRegression()
model_encoded.fit(X_encoded, y)

# Avalie o desempenho do modelo com dummies
score_encoded = model_encoded.score(X_encoded, y)
print(f"Score do modelo com dummies: {score_encoded}")

# Crie uma transformação polinomial de grau 3 na idade
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(X)

# Crie um modelo de regressão linear com a transformação polinomial
model_poly = LinearRegression()
model_poly.fit(X_poly, y)

# Avalie o desempenho do modelo com transformação polinomial
score_poly = model_poly.score(X_poly, y)
print(f"Score do modelo com transformação polinomial: {score_poly}")


Score do modelo original: 0.0028419607345603337
Score do modelo com dummies: 0.008302366577635945
Score do modelo com transformação polinomial: 0.00991266613211117
